In [ ]:
import networkx as nx
import EoN
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from collections import defaultdict
import numpy as np
import random
import pandas as pd
import time
from tqdm import tqdm
from itertools import combinations
import pandas as pd

import matplotlib as mpl
import datetime
import scipy.stats
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator, drange)

import seaborn as sns
mpl.rcParams.update(mpl.rcParamsDefault)
inline_rc = dict(mpl.rcParams)

# functions

In [ ]:
## function for update the inforamation of each node
def update_time(reulst, G, N, upper_bound = 10000):
    for i in range(N):
        if G._node[i]['status'] == "S":
            for j in range(1, len(reulst.node_history(i)[0])):
                if reulst.node_history(i)[0][j] <= upper_bound:
                    G._node[i].update({reulst.node_history(i)[1][j]: 
                                       reulst.node_history(i)[0][j]})
        elif G._node[i]['status'] == "E":
            for j in range(1, len(reulst.node_history(i)[0])):
                if reulst.node_history(i)[0][j] <= upper_bound:
                    G._node[i].update({reulst.node_history(i)[1][j]: 
                                       reulst.node_history(i)[0][j]})
        elif G._node[i]['status'] == "I":
            for j in range(1, len(reulst.node_history(i)[0])):
                if reulst.node_history(i)[0][j] <= upper_bound:
                    G._node[i].update({reulst.node_history(i)[1][j]: 
                                       reulst.node_history(i)[0][j]})
                                   
def update_status(G, N):
    for i in range(N):
        if G._node[i]['R']> -0.1:
            G._node[i].update({'status': "R"})
        elif G._node[i]['I']> - 0.1:
            G._node[i].update({'status': "I"})
        elif G._node[i]['E'] > -0.1:
             G._node[i].update({'status': "E"})
            
def update_from(result, G, upper_bound = 10000):
    for i in range(len(result.transmissions())):
        j = result.transmissions()[i][2]
        if G._node[j]['From'] == -1:
            if result.transmissions()[i][0]<= upper_bound:
                G._node[j].update({'From': result.transmissions()[i][1]})

In [ ]:
## function to analyze the list
def mean_var_std(list_aim):
    return(round(np.mean(list_aim),3),
           round(np.var(list_aim),3),
           round(np.std(list_aim),3))


In [ ]:
 def N_average_move(input_list, average_num):
    cumsum = [0]
    if average_num == 3:
        moving_aves = [sum(input_list[:2])/2]
    if average_num == 5:
        moving_aves = [sum(input_list[:3])/3]
        moving_aves.append(sum(input_list[:4])/4)
    if average_num == 7:
        moving_aves = [sum(input_list[:4])/4]
        moving_aves.append(sum(input_list[:5])/5)
        moving_aves.append(sum(input_list[:6])/6)
    if average_num == 9:
        moving_aves = [sum(input_list[:5])/5]
        moving_aves.append(sum(input_list[:6])/6)
        moving_aves.append(sum(input_list[:7])/7)
        moving_aves.append(sum(input_list[:8])/8)
    for i, x in enumerate(input_list, 1):
        cumsum.append(cumsum[i-1] + x)
        if i>=average_num:
            moving_ave = (cumsum[i] - cumsum[i-average_num])/average_num
            #can do stuff with moving_ave here
            moving_aves.append(moving_ave)
    if average_num == 3:
        moving_aves.append(sum(input_list[-2:])/2)
    elif average_num == 5:
        moving_aves.append(sum(input_list[-4:])/4)
        moving_aves.append(sum(input_list[-3:])/3)
    elif average_num == 7:
        moving_aves.append(sum(input_list[-6:])/6)
        moving_aves.append(sum(input_list[-5:])/5)
        moving_aves.append(sum(input_list[-4:])/4)
    else:
        moving_aves.append(sum(input_list[-8:])/8)
        moving_aves.append(sum(input_list[-7:])/7)
        moving_aves.append(sum(input_list[-6:])/6)
        moving_aves.append(sum(input_list[-5:])/5)
    
    return(moving_aves)

In [ ]:
## plot the figure
def plot_mean_uppper_lower (fgure_for_plot, t_list_stand, S_list_stand, I_list_stand, finished_time, social_apply_time):
    
    S_mean = [np.mean(S_list_stand[i][1:]) for i in range(len(S_list_stand))]
    S_lowerbound  = [np.quantile(S_list_stand[i][1:], 0.05) for i in range(len(S_list_stand))]
    S_upperbound  = [np.quantile(S_list_stand[i][1:], 0.95) for i in range(len(S_list_stand))]
    I_mean = [np.mean(I_list_stand[i][1:]) for i in range(len(I_list_stand))]

    fi_in =t_list_stand.index(min(t_list_stand,key=lambda x:abs(x-finished_time)))

    fgure_for_plot.plot(t_list_stand[:fi_in], S_mean[:fi_in], color = 'C0')
    fgure_for_plot.fill_between(t_list_stand[:fi_in], S_lowerbound[:fi_in],S_upperbound[:fi_in], facecolor = 'C0', alpha=0.5,label='90% CI Total Confimred Cases' )
    fgure_for_plot.scatter(t_list_stand[fi_in],  S_mean[fi_in], color ='C0', label = 'Total Confimred Cases \n {} nodes (Mean)'.format(round( S_mean[fi_in],2)))
    fgure_for_plot.axvline(x= np.median(social_apply_time), color= 'black', linestyle = '--', label = 'Social Distancing Starts at \n  {} days (Mean)'.format(round(social_apply_time,2)))


In [ ]:
def G_for_country(number_node, num_neigbor, p_newman_po, family_size):
    neigbor = num_neigbor
    p_newman = p_newman_po
    print("generating graph G with {} nodes {} neigbor and {} newconnect".
          format(number_node, neigbor, p_newman))

    G_tem = nx.generators.random_graphs.watts_strogatz_graph(number_node, num_neigbor, p_newman_po)

    for i in range(number_node):
        G_tem._node[i].update({'name':i})
        G_tem._node[i].update({'status':'S'})
        G_tem._node[i].update({'From': -1})
        G_tem._node[i].update({'S': 0})
        G_tem._node[i].update({'E': -1})
        G_tem._node[i].update({'I': -1})
        G_tem._node[i].update({'R': -1})
    
    country_stable_edges_list = []
    country_long_dis_edges_list = []
    for i in G_tem.edges:
        tem_index = int(i[0]/family_size)
        if i[1] in  range(tem_index*family_size,tem_index*family_size+family_size):
            country_stable_edges_list.append(i)
        else:
            country_long_dis_edges_list.append(i)

    print("num orginal edges: ", len(G_tem.edges))
    print("num baxic edges: ", len(country_stable_edges_list))
    print("num long dis edges: ", len(country_long_dis_edges_list))
    
    return(G_tem,country_stable_edges_list, country_long_dis_edges_list )

In [ ]:
# simulation with the end of the second stage
def SEIR_only_social_distancing(G,infected_rate, recoverey_rate,try_end_stage_one, 
                                stage_one_threshold, G_remove_list, max_time_simulation,quaran='F'):
    # the whole recored
    combine_t = []
    combine_I = []
    combine_E = []
    combine_S = []
    combine_R = []
    return_statuses = ('S', 'E', 'I', 'R')
    
    infrate = 0.019
    # build the subgraph
    H_pre = nx.DiGraph()
    H_pre.add_node('S')
    H_pre.add_edge('E', 'I', rate = infected_rate,)
    H_pre.add_edge('I', 'R', rate = recoverey_rate)
    J_pre = nx.DiGraph()
    J_pre.add_edge(('I', 'S'), ('I', 'E'), rate = infrate)
    J_pre.add_edge(('E', 'S'), ('E', 'E'), rate = infrate)
    
    
    # set inital input variables
    start_time = 0
    end_time = try_end_stage_one 
    inital_status = nx.get_node_attributes(G, 'status')
    num_node = len(G._node)
    
    final_cuc = 0
    while final_cuc < round(stage_one_threshold*num_node)+1:
        # run simulation in each time_interval before the first time threshold coming
        sim_data =  EoN.Gillespie_simple_contagion(G, H_pre, J_pre, inital_status, return_statuses,
                                                   tmin = start_time, tmax =end_time,return_full_data=True)
        tem_cuc_list = [sim_data.I().tolist()[k]+sim_data.R().tolist()[k] for k in range(len(sim_data.I()))]
        final_cuc = tem_cuc_list[-1]
    
    close_stage_1_time_index = tem_cuc_list.index(round(stage_one_threshold*num_node))+1
    close_stage_1_time = sim_data.t()[close_stage_1_time_index-1]
    
    ## updata information of each node
    update_time(sim_data, G, num_node, close_stage_1_time)
    update_status(G, num_node)
    update_from(sim_data ,G,close_stage_1_time)
    
    ## record the simiulation result
    combine_t = combine_t + sim_data.t().tolist()[:close_stage_1_time_index]
    combine_E = combine_E + sim_data.summary()[1]['E'].tolist()[:close_stage_1_time_index]
    combine_S = combine_S + sim_data.S().tolist()[:close_stage_1_time_index]
    combine_I = combine_I + sim_data.I().tolist()[:close_stage_1_time_index]
    combine_R = combine_R + sim_data.R().tolist()[:close_stage_1_time_index]
    G_1 =G.copy()
    E_case_in_sd = combine_E[-1]


    
    ## The second stage
    # Remove list from remove_list  
    for i in G_remove_list:
        G.remove_edge(*i)
    
    inital_status = nx.get_node_attributes(G, 'status')
    start_time = close_stage_1_time
    
    #if quaran == "T":
     #   J_pre.edges[(('E', 'S'), ('E', 'E'))]['rate']=0.032
      #  J_pre.edges[(('I', 'S'), ('I', 'E'))]['rate']=0.032
        
    # run the simulation of the second stage after removing edges
    sim_data =  EoN.Gillespie_simple_contagion(G, H_pre, J_pre, inital_status, return_statuses, tmin = start_time, 
                                               tmax = max_time_simulation, return_full_data = True)
    
    # updata information of each node
    update_time(sim_data, G, num_node)
    update_status(G, num_node)
    update_from(sim_data ,G)

    # record the simiulation result
    combine_t = combine_t + sim_data.t().tolist()
    combine_S = combine_S + sim_data.S().tolist()
    combine_E = combine_E + sim_data.summary()[1]['E'].tolist()
    combine_I = combine_I + sim_data.I().tolist()
    combine_R = combine_R + sim_data.R().tolist()
    
    return(combine_t,combine_S, combine_E, combine_I,combine_R,close_stage_1_time, G_1, G,sim_data, E_case_in_sd )   


In [ ]:
def SEIR_social_case_multi_run(G_tem, country_stable_edges_list, country_long_dis_edges_list, threshold_social_dis,
                              policy_power, name, repeat_time, max_time_for_simulation):
    # recored each iteration and the time of new infected node
    wo_final_cumu = []
    wo_infected_peak = [] 
    wo_infected_peak_time = []
    wo_final_stop_time = []
    wo_apply_social = []
    wo_exposed_number_in_social = []

    arr_record_cumu = []
    arr_record_I = []
    arr_record_R = []

    label_plot = ['I', 'R','Cumu']
    
    ## create a same timeline for all simualtions of benchmark
    pre_t_standard = [d for d in np.arange(0, max_time_for_simulation+0.2, 0.2)]
    arr_record_cumu = []
    arr_record_I = []
    arr_record_R = []
    
    ## recored all run result in each time point
    for i in pre_t_standard:
        arr_record_cumu.append([i])
        arr_record_I.append([i])
        arr_record_R.append([i])

    # reocrd the time
    for i in tqdm(range(repeat_time)):
        G = G_tem.copy()
        N = len(G_tem.nodes())
        initial_infected_list = random.sample(range(N), round(N*rho_set))
        for i in initial_infected_list:
            G._node[i]['I'] = 0
            G._node[i]['status'] = 'I'
            G._node[i]['From'] = None

        prob_edges_remove =policy_power
        # random pick remove list from non-neighbor edages   
        G_country_remove_list = random.sample(country_long_dis_edges_list, 
                                              round(len(country_long_dis_edges_list)*prob_edges_remove))

        result = SEIR_only_social_distancing(G,tau ,gamma,30,threshold_social_dis, 
                                             G_country_remove_list, max_time_for_simulation,quaran="T")

        pre_t = result[0]
        pre_S = result[1]
        pre_I = result[3]
        pre_R = result[4]
        result_stage = result[5]
        
        infected_peak = max(pre_I)
        infected_peak_time = pre_t[pre_I.index(infected_peak)]
        final_stop_time  = pre_t[-1]

        # combine all data following the time line
        index_time_line = [pre_t.index(min(pre_t, key=lambda x:abs(x-i))) for i in pre_t_standard] 
        pre_I_standard = [pre_I[i] for i in index_time_line]
        pre_R_standard = [pre_R[i] for i in index_time_line]

        for i in range(len(pre_t_standard)):
            arr_record_cumu[i].append(pre_I_standard[i]+pre_R_standard[i])
            arr_record_I[i].append(pre_I_standard[i])
            arr_record_R[i].append(pre_R_standard[i])

        wo_final_cumu.append(pre_I[-1]+pre_R[-1] ) 
        wo_infected_peak.append(infected_peak)
        wo_infected_peak_time.append(infected_peak_time) 
        wo_final_stop_time.append(final_stop_time)
        wo_apply_social.append(result_stage)
        wo_exposed_number_in_social.append(result[9])


    fig = plt.figure(figsize = (6,4))
    ax = fig.add_subplot(111)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plot_mean_uppper_lower (ax, pre_t_standard, arr_record_cumu, arr_record_I, 
                            max_time_for_simulation,np.mean(wo_apply_social))

    
    ax.set_xlabel("Time",fontsize=14)
    ax.set_ylabel("Cases",fontsize=14)
    ax.set_title("{}: apply {} policy power when {}% cumulative cases".format(name,round(policy_power*100), round(threshold_social_dis*100,2)))
    #plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    plt.legend(loc='lower right',fontsize = 11)
    plt.tight_layout()
    plt.show()
    
    print("The cumulative case: ", mean_var_std(wo_final_cumu))
    print("The infected peak: ", mean_var_std(wo_infected_peak))
    print("The time of infected peak:", mean_var_std(wo_infected_peak_time))
    print("The time of policy application:", mean_var_std(wo_apply_social))
    print("The exposed case when policy application:", mean_var_std(wo_exposed_number_in_social))
        
    return(pre_t_standard, arr_record_cumu, arr_record_I,wo_apply_social,wo_exposed_number_in_social)


In [ ]:
# simulation without the NPI
def SEIR_only_social_distancing_ps(G,infected_rate, recoverey_rate, max_time_simulation,quaran='F'):
    # the whole recored
    combine_t = []
    combine_I = []
    combine_E = []
    combine_S = []
    combine_R = []
    return_statuses = ('S', 'E', 'I', 'R')
    
    infrate = 0.019
    # build the subgraph
    H_pre = nx.DiGraph()
    H_pre.add_node('S')
    H_pre.add_edge('E', 'I', rate = infected_rate)
    H_pre.add_edge('I', 'R', rate = recoverey_rate)
    J_pre = nx.DiGraph()
    J_pre.add_edge(('I', 'S'), ('I', 'E'), rate = infrate)
    J_pre.add_edge(('E', 'S'), ('E', 'E'), rate = infrate)
    
    
    # set inital input variables
    num_node = len(G._node)       
    inital_status = nx.get_node_attributes(G, 'status')
    start_time = 0
    
        
    # run the simulation of the second stage after removing edges
    sim_data =  EoN.Gillespie_simple_contagion(G, H_pre, J_pre, inital_status, return_statuses, tmin = start_time, 
                                               tmax = max_time_simulation, return_full_data = True)
    
    # updata information of each node
    update_time(sim_data, G, num_node)
    update_status(G, num_node)
    update_from(sim_data ,G)

    # record the simiulation result
    combine_t = combine_t + sim_data.t().tolist()
    combine_S = combine_S + sim_data.S().tolist()
    combine_E = combine_E + sim_data.summary()[1]['E'].tolist()
    combine_I = combine_I + sim_data.I().tolist()
    combine_R = combine_R + sim_data.R().tolist()
    
    return(combine_t,combine_S, combine_E, combine_I,combine_R, G,sim_data )   


In [ ]:
def SEIR_social_case_multi_run_ps(G_tem, country_stable_edges_list, country_long_dis_edges_list, 
                                  name, repeat_time, max_time_for_simulation):
    # recored each iteration and the time of new infected node
    wo_final_cumu = []
    wo_infected_peak = [] 
    wo_infected_peak_time = []
    wo_final_stop_time = []

    arr_record_cumu = []
    arr_record_I = []
    arr_record_R = []

    label_plot = ['I', 'R','Cumu']
    
    ## create a same timeline for all simualtions of benchmark
    pre_t_standard = [d for d in np.arange(0, max_time_for_simulation+0.2, 0.2)]
    arr_record_cumu = []
    arr_record_I = []
    arr_record_R = []
    
    ## recored all run result in each time point
    for i in pre_t_standard:
        arr_record_cumu.append([i])
        arr_record_I.append([i])
        arr_record_R.append([i])

    # reocrd the time
    for i in tqdm(range(repeat_time)):
        G = G_tem.copy()
        N = len(G_tem.nodes())
        initial_infected_list = random.sample(range(N), round(N*rho_set))
        for i in initial_infected_list:
            G._node[i]['I'] = 0
            G._node[i]['status'] = 'I'
            G._node[i]['From'] = None


        result = SEIR_only_social_distancing_ps(G,tau ,gamma,max_time_for_simulation,quaran="F")

        pre_t = result[0]
        pre_S = result[1]
        pre_I = result[3]
        pre_R = result[4]
        
        infected_peak = max(pre_I)
        infected_peak_time = pre_t[pre_I.index(infected_peak)]
        final_stop_time  = pre_t[-1]

        # combine all data following the time line
        index_time_line = [pre_t.index(min(pre_t, key=lambda x:abs(x-i))) for i in pre_t_standard] 
        pre_I_standard = [pre_I[i] for i in index_time_line]
        pre_R_standard = [pre_R[i] for i in index_time_line]

        for i in range(len(pre_t_standard)):
            arr_record_cumu[i].append(pre_I_standard[i]+pre_R_standard[i])
            arr_record_I[i].append(pre_I_standard[i])
            arr_record_R[i].append(pre_R_standard[i])

        wo_final_cumu.append(pre_I[-1]+pre_R[-1] ) 
        wo_infected_peak.append(infected_peak)
        wo_infected_peak_time.append(infected_peak_time) 
        wo_final_stop_time.append(final_stop_time)

    
    print("The cumulative case: ", mean_var_std(wo_final_cumu))
    print("The infected peak: ", mean_var_std(wo_infected_peak))
    print("The time of infected peak:", mean_var_std(wo_infected_peak_time))
        
    return(pre_t_standard, arr_record_cumu, arr_record_I)


In [ ]:
# simulation with the end of the second stage
def SEIR_only_social_distancing_ps2(G,infected_rate, recoverey_rate,try_end_stage_one, G_remove_list, max_time_simulation,quaran='F'):
    # the whole recored
    combine_t = []
    combine_I = []
    combine_E = []
    combine_S = []
    combine_R = []
    return_statuses = ('S', 'E', 'I', 'R')
    
    infrate = 0.019
    # build the subgraph
    H_pre = nx.DiGraph()
    H_pre.add_node('S')
    H_pre.add_edge('E', 'I', rate = infected_rate)
    H_pre.add_edge('I', 'R', rate = recoverey_rate)
    J_pre = nx.DiGraph()
    J_pre.add_edge(('I', 'S'), ('I', 'E'), rate = infrate)
    J_pre.add_edge(('E', 'S'), ('E', 'E'), rate = infrate)
    
    
    # set inital input variables
    start_time = 0
    end_time = try_end_stage_one 
    inital_status = nx.get_node_attributes(G, 'status')
    num_node = len(G._node)
    

    sim_data =  EoN.Gillespie_simple_contagion(G, H_pre, J_pre, inital_status, return_statuses,
                                                   tmin = start_time, tmax =end_time,return_full_data=True)

    close_stage_1_time = try_end_stage_one 
    ## updata information of each node
    update_time(sim_data, G, num_node, try_end_stage_one )
    update_status(G, num_node)
    update_from(sim_data ,G,try_end_stage_one)
    
    ## record the simiulation result
    combine_t = combine_t + sim_data.t().tolist()
    combine_E = combine_E + sim_data.summary()[1]['E'].tolist()
    combine_S = combine_S + sim_data.S().tolist()
    combine_I = combine_I + sim_data.I().tolist()
    combine_R = combine_R + sim_data.R().tolist()
    G_1 =G.copy()
    E_case_in_sd = combine_E[-1]


    
    ## The second stage
    # Remove list from remove_list  
    for i in G_remove_list:
        G.remove_edge(*i)
    
    inital_status = nx.get_node_attributes(G, 'status')
    start_time = close_stage_1_time
    
    #if quaran == "T":
     #   J_pre.edges[(('E', 'S'), ('E', 'E'))]['rate']=0.032
      #  J_pre.edges[(('I', 'S'), ('I', 'E'))]['rate']=0.032
        
    # run the simulation of the second stage after removing edges
    sim_data =  EoN.Gillespie_simple_contagion(G, H_pre, J_pre, inital_status, return_statuses, tmin = start_time, 
                                               tmax = max_time_simulation, return_full_data = True)
    
    # updata information of each node
    update_time(sim_data, G, num_node)
    update_status(G, num_node)
    update_from(sim_data ,G)

    # record the simiulation result
    combine_t = combine_t + sim_data.t().tolist()
    combine_S = combine_S + sim_data.S().tolist()
    combine_E = combine_E + sim_data.summary()[1]['E'].tolist()
    combine_I = combine_I + sim_data.I().tolist()
    combine_R = combine_R + sim_data.R().tolist()
    
    return(combine_t,combine_S, combine_E, combine_I,combine_R,close_stage_1_time, G_1, G,sim_data, E_case_in_sd )   


In [ ]:
def SEIR_social_case_multi_run_ps2(G_tem, country_stable_edges_list, country_long_dis_edges_list, threshold_time,
                              policy_power, name, repeat_time, max_time_for_simulation):
    # recored each iteration and the time of new infected node
    wo_final_cumu = []
    wo_infected_peak = [] 
    wo_infected_peak_time = []
    wo_final_stop_time = []
    wo_apply_social = []
    wo_exposed_number_in_social = []

    arr_record_cumu = []
    arr_record_I = []
    arr_record_R = []

    label_plot = ['I', 'R','Cumu']
    
    ## create a same timeline for all simualtions of benchmark
    pre_t_standard = [d for d in np.arange(0, max_time_for_simulation+0.2, 0.2)]
    arr_record_cumu = []
    arr_record_I = []
    arr_record_R = []
    
    ## recored all run result in each time point
    for i in pre_t_standard:
        arr_record_cumu.append([i])
        arr_record_I.append([i])
        arr_record_R.append([i])

    # reocrd the time
    for i in tqdm(range(repeat_time)):
        G = G_tem.copy()
        N = len(G_tem.nodes())
        initial_infected_list = random.sample(range(N), round(N*rho_set))
        for i in initial_infected_list:
            G._node[i]['I'] = 0
            G._node[i]['status'] = 'I'
            G._node[i]['From'] = None

        prob_edges_remove =policy_power
        # random pick remove list from non-neighbor edages   
        G_country_remove_list = random.sample(country_long_dis_edges_list, 
                                              round(len(country_long_dis_edges_list)*prob_edges_remove))

        result = SEIR_only_social_distancing_ps2(G,tau ,gamma,threshold_time,
                                             G_country_remove_list, max_time_for_simulation,quaran="T")

        pre_t = result[0]
        pre_S = result[1]
        pre_I = result[3]
        pre_R = result[4]
        result_stage = result[5]
        
        infected_peak = max(pre_I)
        infected_peak_time = pre_t[pre_I.index(infected_peak)]
        final_stop_time  = pre_t[-1]

        # combine all data following the time line
        index_time_line = [pre_t.index(min(pre_t, key=lambda x:abs(x-i))) for i in pre_t_standard] 
        pre_I_standard = [pre_I[i] for i in index_time_line]
        pre_R_standard = [pre_R[i] for i in index_time_line]

        for i in range(len(pre_t_standard)):
            arr_record_cumu[i].append(pre_I_standard[i]+pre_R_standard[i])
            arr_record_I[i].append(pre_I_standard[i])
            arr_record_R[i].append(pre_R_standard[i])

        wo_final_cumu.append(pre_I[-1]+pre_R[-1] ) 
        wo_infected_peak.append(infected_peak)
        wo_infected_peak_time.append(infected_peak_time) 
        wo_final_stop_time.append(final_stop_time)
        wo_apply_social.append(result_stage)
        wo_exposed_number_in_social.append(result[9])

    
    print("The cumulative case: ", mean_var_std(wo_final_cumu))
    print("The infected peak: ", mean_var_std(wo_infected_peak))
    print("The time of infected peak:", mean_var_std(wo_infected_peak_time))
    print("The time of policy application:", mean_var_std(wo_apply_social))
    print("The exposed case when policy application:", mean_var_std(wo_exposed_number_in_social))
        
    return(pre_t_standard, arr_record_cumu, arr_record_I,wo_apply_social,wo_exposed_number_in_social)


# Parameters setting

In [ ]:
## number of nodes
N =10000 

# initial infected fraction, 1 agent
rho_set = 0.0001

#transmission rate
tau = 1/7
# the expected connecting time with neighbor before infected is 1/tau  
# the variance is 1/(tau*tau)

#recovery rate
gamma = 1/14
# the expected recovered time after infected is 1/gamma 
# the varance is 1/(gamam*gamma)

# Contact-based Networking Building 

In [ ]:
## For each country, the contact-based network is constructed by the number of the mean reported contacts and the average household size

In [ ]:
# Germany 
# 7.96 mean reported contacts and 2.05 average household size
G_Germany, Germany_stable_edges_list, Germany_long_dis_edges_list = G_for_country(N, 8, 0.41,3)

G_play = G_Germany.copy()
for i in Germany_long_dis_edges_list:
    G_play.remove_edge(*i)
G_subgraph = [i for i in nx.connected_components(G_play)]
G_subgraph_node_list  = [len(i) for i in G_subgraph]
print("The distribution of household size")
print(mean_var_std(G_subgraph_node_list))


In [ ]:
# Ialty
# 19.77 mean reported contacts and 2.40 average household size
G_It, It_stable_edges_list, It_long_dis_edges_list =  G_for_country(N, 20, 0.31,3)

G_play = G_It.copy()
for i in  It_long_dis_edges_list:
    G_play.remove_edge(*i)
G_subgraph = [i for i in nx.connected_components(G_play)]
G_subgraph_node_list  = [len(i) for i in G_subgraph]
print("The distribution of household size")
print(mean_var_std(G_subgraph_node_list))

In [ ]:
# Nethelands
# 13.85 mean reported contacts and 2.23 average household size
G_Neth, Neth_stable_edges_list, Neth_long_dis_edges_list = G_for_country(N, 14, 0.36,3)
G_play = G_Neth.copy()
for i in  Neth_long_dis_edges_list:
    G_play.remove_edge(*i)
G_subgraph = [i for i in nx.connected_components(G_play)]
G_subgraph_node_list  = [len(i) for i in G_subgraph]
print("The distribution of household size")
print(mean_var_std(G_subgraph_node_list))

In [ ]:
# Belgium
# 11.84 mean reported contacts and 2.36 average household size
G_Be, Be_stable_edges_list, Be_long_dis_edges_list =  G_for_country(N, 12, 0.32,3)
G_play = G_Be.copy()
for i in  Be_long_dis_edges_list:
    G_play.remove_edge(*i)
G_subgraph = [i for i in nx.connected_components(G_play)]
G_subgraph_node_list  = [len(i) for i in G_subgraph]
print("The distribution of household size")
print(mean_var_std(G_subgraph_node_list))

In [ ]:
## UK 
# 11.74 mean reported contacts and 2.27 average household size
G_UK, UK_stable_edges_list, UK_long_dis_edges_list = G_for_country(N, 12,0.35,3)
    
G_play = G_UK.copy()
for i in UK_long_dis_edges_list:
    G_play.remove_edge(*i)
G_subgraph = [i for i in nx.connected_components(G_play)]
G_subgraph_node_list  = [len(i) for i in G_subgraph]
print("The distribution of household size")
print(mean_var_std(G_subgraph_node_list))

#  Simulatie the pandenic of countries in the first 60 days. All modified SEIR models using in the following simulaiton are shown in the function section.

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

In [ ]:
plt.style.use('seaborn-white')

In [ ]:
## number of run
tst = 500
## time of each run
tim  = 60
Be_social_dis = SEIR_social_case_multi_run(G_Be, Be_stable_edges_list, Be_long_dis_edges_list, 0.0032,
                              0.8148,'Belgium', tst ,tim)

It_social_dis = SEIR_social_case_multi_run(G_It, It_stable_edges_list, It_long_dis_edges_list, 0.0016,
                              0.9352,'Italy', tst ,tim)

UK_social_dis = SEIR_social_case_multi_run(G_UK, UK_stable_edges_list, UK_long_dis_edges_list, 0.0015,
                              0.7963,'United Kingdom', tst , tim)

neth_social_dis = SEIR_social_case_multi_run(G_Neth, Neth_stable_edges_list, Neth_long_dis_edges_list, 0.0011,
                              0.7963,'Netherlands', tst , tim)

Germany_social_dis= SEIR_social_case_multi_run(G_Germany, Germany_stable_edges_list, Germany_long_dis_edges_list, 0.0025,
                              0.7685,'Germany', tst , tim)


# Data generation for Scenario-based Analysis

In [ ]:
## number of nodes
N =10000 
# initial infected fraction, 1 agent
rho_set = 0.001
#transmission rate
tau = 1/7
#recovery rate
gamma = 1/14
## number of run
tst = 200
## time of each run
tim  = 100
ax_poli = fig.add_subplot(131)
HH_ps = [1.9, 2.9, 3.9]
p_ps  = [0.65,0.44, 0.18]

NPI_str = 0.9
saved_G_HH42 = []
saved_GHH42_stable_edges_list= []
saved_HH42_long_dis_edges_list = []

## Early NPIs
NPI_time = 10
for HH_picked in HH_ps :
    tin = HH_ps.index(HH_picked)
    G_HH42, HH42_stable_edges_list,HH42_long_dis_edges_list = G_for_country(N,10, p_ps[tin],4)
    saved_G_HH42.append(G_HH42)
    saved_GHH42_stable_edges_list.append(HH42_stable_edges_list)
    saved_HH42_long_dis_edges_list.append(HH42_long_dis_edges_list)
    ## Show the netwrok structure
    G_play = G_HH42.copy()
    G_play_2 = G_HH42.copy()
    for i in HH42_long_dis_edges_list:
        G_play.remove_edge(*i)
    G_subgraph = [i for i in nx.connected_components(G_play)]
    G_subgraph_node_list  = [len(i) for i in G_subgraph]
    print("The distribution of household size: ", mean_var_std(G_subgraph_node_list))
    ## simulation
    test = SEIR_social_case_multi_run_ps2(G_play_2, HH42_stable_edges_list, HH42_long_dis_edges_list, NPI_time,NPI_str,'test', tst ,tim)

    np.save("ea_npi"+level_n[tin], test)
    
## Late NPIs
NPI_time = 25    
for HH_picked in HH_ps :
    tin = HH_ps.index(HH_picked)
    ## used the generated network above
    G_HH42 = saved_G_HH42[tin]
    HH42_stable_edges_list = saved_GHH42_stable_edges_list[tin]
    HH42_long_dis_edges_list = saved_HH42_long_dis_edges_list[tin]
    G_play_2 = G_HH42.copy()
    ## simulation
    test = SEIR_social_case_multi_run_ps2(G_play_2, HH42_stable_edges_list, HH42_long_dis_edges_list, NPI_time,NPI_str,'test', tst ,tim)
    np.save("la_npi"+level_n[tin], test)

    

## No NPIs
for HH_picked in HH_ps :
    tin = HH_ps.index(HH_picked)
    ## used the generated network above
    G_HH42 = saved_G_HH42[tin]
    HH42_stable_edges_list = saved_GHH42_stable_edges_list[tin]
    HH42_long_dis_edges_list = saved_HH42_long_dis_edges_list[tin]
    G_play_2 = G_HH42.copy()
    ## simulation
    test = SEIR_social_case_multi_run_ps(G_play_2 , HH42_stable_edges_list, HH42_long_dis_edges_list, 'test', tst ,tim)
    np.save("ea_no_npi"+level_n[tin], test)


In [ ]:
## number of nodes
N =10000 
# initial infected fraction, 1 agent
rho_set = 0.001
#transmission rate
tau = 1/7
#recovery rate
gamma = 1/14

tst = 200
## time of each run
tim  = 100
## the setting of t_NPI
M_ps = [4+0.5*i for i in range(55)]
NPI_str = 0.9

G_HH42, HH42_stable_edges_list,HH42_long_dis_edges_list = G_for_country(N, 10, 0.65,4)
## Show the netwrok structure
G_play = G_HH42.copy() 
for i in HH42_long_dis_edges_list:
    G_play.remove_edge(*i)
G_subgraph = [i for i in nx.connected_components(G_play)]
G_subgraph_node_list  = [len(i) for i in G_subgraph]
print("The distribution of household size: ", mean_var_std(G_subgraph_node_list))

low_ratio_re_505 = []
low_ratio_re_1005 = []
## simulation 
for M_picked in M_ps :
    G_play_2 =G_HH42.copy()
    test = SEIR_social_case_multi_run_ps2(G_play_2, HH42_stable_edges_list, HH42_long_dis_edges_list, M_picked, NPI_str,'test', tst ,tim)
    
    low_ratio_re_505.append(test[1][-51*5][1:])
    low_ratio_re_1005.append(test[1][-1][1:])

np.save("low_ratio_npi_505", low_ratio_re_505)
np.save("low_ratio_npi_1005", low_ratio_re_1005)

G_HH42, HH42_stable_edges_list,HH42_long_dis_edges_list = G_for_country(N, 10, 0.18,4)

## Show the netwrok structure
G_play = G_HH42.copy() 
for i in HH42_long_dis_edges_list:
    G_play.remove_edge(*i)
G_subgraph = [i for i in nx.connected_components(G_play)]
G_subgraph_node_list  = [len(i) for i in G_subgraph]
print("The distribution of household size: ", mean_var_std(G_subgraph_node_list))

high_ratio_re_505 = []
high_ratio_re_1005 = []

## simulation 
for M_picked in M_ps :
    G_play_2 =G_HH42.copy()
    test = SEIR_social_case_multi_run_ps2(G_play_2, HH42_stable_edges_list, HH42_long_dis_edges_list, M_picked, NPI_str,'test', tst ,tim)
    high_ratio_re_505.append(test[1][-51*5][1:])
    high_ratio_re_1005.append(test[1][-1][1:])
    
np.save("high_ratio_npi_505", high_ratio_re_505)
np.save("high_ratio_npi_1005", high_ratio_re_1005)

# Counterfactual Analysis

# Data generation

In [ ]:
## number of run
tst = 200
## time of each run
tim  = 60
Be_conclusion = [[],[],[],[],[],[]]
Be_conclusion_name = ['BE','BE_IT', 'BE_UK','BE_NL','BE_DE']
Be_conclusion[0] = SEIR_social_case_multi_run(G_Be, Be_stable_edges_list, Be_long_dis_edges_list, 0.0032,
                              0.8148,'Belgium', tst ,tim)
Be_conclusion[1]  = SEIR_social_case_multi_run(G_Be, Be_stable_edges_list, Be_long_dis_edges_list, 0.0016,
                              0.9352,'Beligum_Italy', tst ,tim)
Be_conclusion[2] = SEIR_social_case_multi_run(G_Be, Be_stable_edges_list, Be_long_dis_edges_list, 0.0015,
                              0.7963,'Beligum_United Kingdom', tst , tim)
Be_conclusion[3] = SEIR_social_case_multi_run(G_Be, Be_stable_edges_list, Be_long_dis_edges_list, 0.0011,
                              0.7963,'Beligum_Netherlands', tst , tim)
Be_conclusion[4]  = SEIR_social_case_multi_run(G_Be, Be_stable_edges_list, Be_long_dis_edges_list, 0.0025,
                              0.7685,'Beligum_Germany', tst , tim)

In [ ]:
## number of run
tst = 200
## time of each run
tim  = 60
DE_conclusion = [[],[],[],[],[],[]]
DE_conclusion_name = ['DE_BE','DE_IT', 'DE_UK','DE_NL','DE']
DE_conclusion[0] = SEIR_social_case_multi_run(G_Germany, Germany_stable_edges_list, Germany_long_dis_edges_list, 0.0032,
                              0.8148,'Germany_Belgium', tst ,tim)
DE_conclusion[1] = SEIR_social_case_multi_run(G_Germany, Germany_stable_edges_list, Germany_long_dis_edges_list, 0.0016,
                              0.9352,'Germany_Italy', tst ,tim)
DE_conclusion[2]= SEIR_social_case_multi_run(G_Germany, Germany_stable_edges_list, Germany_long_dis_edges_list, 0.0015,
                              0.7963,'Germany_United Kingdom', tst , tim)
DE_conclusion[3] = SEIR_social_case_multi_run(G_Germany, Germany_stable_edges_list, Germany_long_dis_edges_list, 0.0011,
                              0.7963,'Germany_Netherlands', tst , tim)
DE_conclusion[4] = SEIR_social_case_multi_run(G_Germany, Germany_stable_edges_list, Germany_long_dis_edges_list, 0.0025,
                              0.7685,'Germany', tst , tim)

In [ ]:
## number of run
tst = 200
## time of each run
tim  = 60
IT_conclusion = [[],[],[],[],[],[]]
IT_conclusion_name = ['IT_BE','IT', 'IT_UK','IT_NL','IT_DE']
IT_conclusion[0] = SEIR_social_case_multi_run(G_It, It_stable_edges_list, It_long_dis_edges_list, 0.0032,
                              0.8148,'It_Belgium', tst ,tim)
IT_conclusion[1] = SEIR_social_case_multi_run(G_It, It_stable_edges_list, It_long_dis_edges_list, 0.0016,
                              0.9352,'Italy', tst ,tim)
IT_conclusion[2] = SEIR_social_case_multi_run(G_It, It_stable_edges_list, It_long_dis_edges_list, 0.0015,
                              0.7963,'It_United Kingdom', tst , tim)
IT_conclusion[3] = SEIR_social_case_multi_run(G_It, It_stable_edges_list, It_long_dis_edges_list, 0.0011,
                              0.7963,'It_Netherlands', tst , tim)
IT_conclusion[4]= SEIR_social_case_multi_run(G_It, It_stable_edges_list, It_long_dis_edges_list, 0.0025,
                              0.7685,'It_Germany', tst , tim)

In [ ]:
## number of run
tst = 200
## time of each run
tim  = 60
UK_conclusion = [[],[],[],[],[],[]]
UK_conclusion_name = ['UK_BE','UK_IT', 'UK','UK_NL','UK_DE']
UK_conclusion[0] = SEIR_social_case_multi_run(G_UK, UK_stable_edges_list, UK_long_dis_edges_list, 0.0032,
                              0.8148,'UK_Belgium', tst ,tim)
UK_conclusion[1] = SEIR_social_case_multi_run(G_UK, UK_stable_edges_list, UK_long_dis_edges_list, 0.0016,
                              0.9352,'UK_Italy', tst ,tim)
UK_conclusion[2] = SEIR_social_case_multi_run(G_UK, UK_stable_edges_list, UK_long_dis_edges_list, 0.0015,
                              0.7963,'United Kingdom', tst , tim)
UK_conclusion[3] = SEIR_social_case_multi_run(G_UK, UK_stable_edges_list, UK_long_dis_edges_list, 0.0011,
                              0.7963,'UK_Netherlands', tst , tim)
UK_conclusion[4] = SEIR_social_case_multi_run(G_UK, UK_stable_edges_list, UK_long_dis_edges_list, 0.0025,
                              0.7685,'UK_Germany', tst , tim)

In [ ]:
## number of run
tst = 200
## time of each run
tim  = 60
NL_conclusion = [[],[],[],[],[],[]]
NL_conclusion_name = ['NL_BE','NL_IT', 'NL_UK','NL','NL_DE']
NL_conclusion[0] = SEIR_social_case_multi_run(G_Neth, Neth_stable_edges_list, Neth_long_dis_edges_list, 0.0032,
                              0.8148,'NL_Belgium', tst ,tim)
NL_conclusion[1] = SEIR_social_case_multi_run(G_Neth, Neth_stable_edges_list, Neth_long_dis_edges_list, 0.0016,
                              0.9352,'NL_Italy', tst ,tim)
NL_conclusion[2] = SEIR_social_case_multi_run(G_Neth, Neth_stable_edges_list, Neth_long_dis_edges_list, 0.0015,
                              0.7963,'NL_United Kingdom', tst , tim)
NL_conclusion[3] = SEIR_social_case_multi_run(G_Neth, Neth_stable_edges_list, Neth_long_dis_edges_list, 0.0011,
                              0.7963,'Netherlands', tst , tim)
NL_conclusion[4] = SEIR_social_case_multi_run(G_Neth, Neth_stable_edges_list, Neth_long_dis_edges_list, 0.0025,
                              0.7685,'NL_Germany', tst , tim)

In [ ]:
all_conclusion = [Be_conclusion,IT_conclusion,UK_conclusion,NL_conclusion,DE_conclusion]